In [6]:
import json
import os

In [7]:
TEXT_TO_NUM = {
    'Zero': 0,
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5,
    'Six': 6,
    'Seven': 7,
    'Eight': 8,
    'Nine': 9,
}

In [8]:
DATA_FOLDER = 'datasets_not_split/rotated_all' # <-- may have a different name
# assuming this ipynb resides in a folder inside project root directory
DATA_PATH = os.path.join(os.path.dirname(os.path.realpath("__file__")), os.pardir, DATA_FOLDER)

ANN_FOLDER = 'ann'
IMG_FOLDER = 'img'
LABEL_FOLDER = 'labels' # <-- this folder should be created beforehand

def create_txt_file(labels_path: str, file_prefix: str):
    json_path = os.path.join(DATA_PATH, ANN_FOLDER, file_prefix+'.png.json')

    with open(json_path) as f:
        file_content = f.read()
        annotation = json.loads(file_content)

    img_height = annotation['size']['height']
    img_width = annotation['size']['width']

    with open(os.path.join(labels_path, file_prefix+'.txt'), 'w') as f:
        for i, obj in enumerate(annotation['objects']):
            class_num = TEXT_TO_NUM[annotation['objects'][i]['classTitle']]
            first_point, second_point = annotation['objects'][i]['points']['exterior']

            upper_left_corner_x, upper_left_corner_y = min(first_point[0], second_point[0]), min(first_point[1], second_point[1])
            bbox_height, bbox_width = abs(first_point[1] - second_point[1]), abs(first_point[0] - second_point[0])
    
            x_center = upper_left_corner_x + bbox_width // 2
            y_center = upper_left_corner_y + bbox_height // 2
    
            output_arr = [class_num, x_center / img_width, y_center / img_height, bbox_width / img_width, bbox_height / img_height]
            output_str = ' '.join([str(s) for s in output_arr])
            f.write(output_str + '\n')

In [9]:
for file_prefix in list(name.replace('.png', '') for name in os.listdir(os.path.join(DATA_PATH, IMG_FOLDER))):
    create_txt_file(os.path.join(DATA_PATH, LABEL_FOLDER), file_prefix)